# 

In [6]:
import json
json.loads(r'''{"file":"da\"ta/2020/jsons2/twitter-2020.08.27.json", "line":"14301", "created_at":"Thu Aug 27 01:50:30 +0000 2020", "place_type":"city", "full_name":"Guadalajara, Jalisco"}''')

{'file': 'da"ta/2020/jsons2/twitter-2020.08.27.json',
 'line': '14301',
 'created_at': 'Thu Aug 27 01:50:30 +0000 2020',
 'place_type': 'city',
 'full_name': 'Guadalajara, Jalisco'}

In [5]:
import json
import os
import pandas as pd
import sys
# NOTA - cada cuantos archivo json agrupar
err_logger_1 = []
err_logger_2 = []
file_no = 0
year = "2020"
write_no = 0
data = []


with open("./data/full_2020.json") as f:
    for line in f:
        #print(line)
        try:
            data.append(json.loads(fr'''{line}'''))
        except Exception as e:
            err_template = "An exception of type {0} occurred. Arguments:\n{1!r}"
            err_message = err_template.format(type(e).__name__, e.args)
            err_logger_1.append({"operation":"Load json", "detail":"full_2020", "err":err_message})
    #continue


try:
    write_no = write_no + 1
    print("gen_file")
    # Json to dataframe
    df = pd.json_normalize(data)

    # Guardar dataframe completo
    df.to_parquet("./data/full_2020.parquet", engine='auto', compression='snappy')

    # Guardar en csv
    #df.to_csv("/content/sample_data/testing.csv", sep=',', encoding='utf-8')

    data = []
except Exception as e:
    err_template = "An exception of type {0} occurred. Arguments:\n{1!r}"
    err_message = err_template.format(type(e).__name__, e.args)
    err_logger_2.append({"operation":"Dataframe & parquet", "detail":"full_2020", "err":err_message})
    #continue

gen_file


In [18]:
err_logger_1

[{'operation': 'Load json',
  'detail': 'full_2020',
  'err': 'An exception of type JSONDecodeError occurred. Arguments:\n("Expecting \',\' delimiter: line 1 column 158 (char 157)",)'}]

In [19]:
err_logger_2

[]

In [7]:
import pandas as pd
import pyarrow.parquet as pq

In [8]:
df = pd.read_parquet("./data/full_2020.parquet")
df.shape

(27621385, 5)

In [9]:
df.head()

,file,line,created_at,place_type,full_name
0,data/2020/jsons2/twitter-2020.12.17.json,2,Thu Dec 17 00:00:02 +0000 2020,city,"Torreón, Coahuila de Zaragoza"
1,data/2020/jsons2/twitter-2020.12.17.json,5,Thu Dec 17 00:00:02 +0000 2020,city,"Mérida, Yucatán"
2,data/2020/jsons2/twitter-2020.12.17.json,7,Wed Dec 16 18:02:41 +0000 2020,city,"Alvarado, Veracruz de Ignacio de la Llave"
3,data/2020/jsons2/twitter-2020.12.17.json,8,Thu Dec 17 00:00:03 +0000 2020,city,"Cuauhtémoc, Distrito Federal"
4,data/2020/jsons2/twitter-2020.12.17.json,9,Thu Dec 17 00:00:04 +0000 2020,city,"Miguel Hidalgo, Distrito Federal"


In [10]:
df["place_type"].unique()

array(['city', 'poi', 'neighborhood', 'admin', 'country'], dtype=object)

In [4]:
from pyarrow import json
import pyarrow.parquet as pq

table = json.read_json('./data/full_2020.json') 
pq.write_table(table, './data/full_pysp_2020.parquet')

ArrowInvalid: JSON parse error: Column(/created_at) was specified twice in row 2

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("JsonToParquetPysparkExample") \
    .getOrCreate()

json_df = spark.read.json("./data/full_2020.json", multiLine=True,) 
json_df.printSchema()
json_df.write.parquet("./data/full_pysp_2020.parquet")

root
 |-- created_at: string (nullable = true)
 |-- file: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- line: string (nullable = true)
 |-- place_type: string (nullable = true)



AnalysisException: [PATH_ALREADY_EXISTS] Path file:/home/kincaid/Documents/ucags/data_science_master/3_sem/big_data/homeworks/tweets/data/full_pysp_2020.parquet already exists. Set mode as "overwrite" to overwrite the existing path.